# Introduction to Docker

**Learning Objectives**
  * Build and run Docker containers
  * Pull Docker images from Docker Hub and Google Container Registry
  * Push Docker images to Google Container Registry

## Overview

Docker is an open platform for developing, shipping, and running applications. With Docker, you can separate your applications from your infrastructure and treat your infrastructure like a managed application. Docker helps you ship code faster, test faster, deploy faster, and shorten the cycle between writing code and running code.

Docker does this by combining kernel containerization features with workflows and tooling that helps you manage and deploy your applications.

Docker containers can be directly used in Kubernetes, which allows them to be run in the Kubernetes Engine with ease. After learning the essentials of Docker, you will have the skillset to start developing Kubernetes and containerized applications.

## Basic Docker commands

See what docker images you have. 

In [1]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED        SIZE
hello-world                    latest    d1165f221234   7 weeks ago    13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   2 months ago   1.73GB


If this is the first time working with docker you won't have any repositories listed. 

**Note**. If you are running this in an AI Notebook, then you should see a single image `gcr.io/inverting-proxy/agent`. This is the container that is currently running the AI Notebook. 

Let's use `docker run` to pull a docker image called `hello-world` from the public registry. The docker daemon will search for the `hello-world` image, if it doesn't find the image locally, it pulls the image from a public registry called Docker Hub, creates a container from that image, and runs the container for you.

In [2]:
!docker run hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



Now when we look at our docker images we should see `hello-world` there as well.

In [3]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED        SIZE
hello-world                    latest    d1165f221234   7 weeks ago    13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   2 months ago   1.73GB


This is the image pulled from the Docker Hub public registry. The Image ID is in `SHA256` hash format—this field specifies the Docker image that's been provisioned. When the docker daemon can't find an image locally, it will by default search the public registry for the image. Let's run the container again:

Now, if we want to run `docker run hello-world` again, it won't have to download from the container registry.

To see all docker containers running, use `docker ps`.

In [4]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS     NAMES
a813e8fe4ec6   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   59 minutes ago   Up 59 minutes             proxy-agent


There are no running containers. **Note. If you are running this in at AI Notebook, you'll see one container running.**

The `hello-world` containers you ran previously already exited. In order to see all containers, including ones that have finished executing, run docker `ps -a`:

In [5]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS                      PORTS     NAMES
98fa6a482ed5   hello-world                    "/hello"                 6 seconds ago    Exited (0) 5 seconds ago              great_jepsen
b285532856e7   hello-world                    "/hello"                 57 minutes ago   Exited (0) 57 minutes ago             dazzling_tharp
a813e8fe4ec6   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   59 minutes ago   Up 59 minutes                         proxy-agent


This shows you the Container ID, a UUID generated by Docker to identify the container, and more metadata about the run. The container Names are also randomly generated but can be specified with docker run --name [container-name] hello-world.

## Build a Docker container

Let's build a Docker image that's based on a simple node application. 

Open a new text file and write the following. Save the file in a folder called `dockerfiles` and name the file `intro.docker`

```bash
# Use an official Node runtime as the parent image
FROM node:6

# Set the working directory in the container to /app
WORKDIR /app

# Copy the current directory contents into the container at /app
ADD . /app

# Make the container's port 80 available to the outside world
EXPOSE 80

# Run app.js using node when the container launches
CMD ["node", "./src/app.js"]
```

This file instructs the Docker daemon on how to build your image.

The initial line specifies the base parent image, which in this case is the official Docker image for node version 6.
In the second, we set the working (current) directory of the container.
In the third, we add the current directory's contents (indicated by the "." ) into the container.
Then we expose the container's port so it can accept connections on that port and finally run the node command to start the application.

Check out the other [Docker command references](https://docs.docker.com/engine/reference/builder/#known-issues-run) to understand what each line does.

We're going to use this Docker container to run a simple node.js app. Have a look at `app.js`. This is a simple HTTP server that listens on port 80 and returns "Hello World."


Now let's build the image. Note again the "`.`", which means current directory so you need to run this command from within the directory that has the Dockerfile.

The `-t` is to name and tag an image with the `name:tag` syntax. The name of the image is `node-app` and the tag is `0.1`. The tag is highly recommended when building Docker images. If you don't specify a tag, the tag will default to latest and it becomes more difficult to distinguish newer images from older ones. Also notice how each line in the Dockerfile above results in intermediate container layers as the image is built.

In [15]:
!docker build -t node-app:0.1 -f dockerfiles/intro.docker .

Sending build context to Docker daemon  91.65kB
Step 1/5 : FROM node:6
 ---> ab290b853066
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> dc1c34f69a79
Step 3/5 : ADD . /app
 ---> 7050de89e639
Step 4/5 : EXPOSE 80
 ---> Running in 512084ec32fb
Removing intermediate container 512084ec32fb
 ---> 15f2be766976
Step 5/5 : CMD ["node", "./src/app.js"]
 ---> Running in cde4ab60e635
Removing intermediate container cde4ab60e635
 ---> 9ba864117cd6
Successfully built 9ba864117cd6
Successfully tagged node-app:0.1


Let's check that the image has been created correctly. 

In [16]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED                  SIZE
node-app                       0.1       9ba864117cd6   Less than a second ago   884MB
<none>                         <none>    b849727e7f46   6 hours ago              884MB
hello-world                    latest    d1165f221234   7 weeks ago              13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   2 months ago             1.73GB
node                           6         ab290b853066   23 months ago            884MB


You should see a `node-app` repository that was created only seconds ago. 

Notice `node` is the base image and `node-app` is the image you built. You can't remove `node` without removing `node-app` first. The size of the image is relatively small compared to VMs. Other versions of the node image such as `node:slim` and `node:alpine` can give you even smaller images for easier portability. The topic of slimming down container sizes is further explored in Advanced Topics. You can view all versions in the official repository here.

Note, you can remove an image from your docker images using `docker rmi [repository]:[tag]`.

## Run a Docker container

Now we'll run the container based on the image you built above using the `docker run` command. The `--name` flag allows you to name the container if you like. And `-p` instructs Docker to map the host's port 4000 to the container's port 80. This allows you to reach the server at http://localhost:4000. Without port mapping, you would not be able to reach the container at localhost.

In [17]:
!docker ps -a

CONTAINER ID   IMAGE                          COMMAND                  CREATED       STATUS                   PORTS                  NAMES
7ca2f6f75d5b   b849727e7f46                   "node ./src/app.js"      6 hours ago   Up 6 hours               0.0.0.0:4000->80/tcp   my-app
98fa6a482ed5   hello-world                    "/hello"                 6 hours ago   Exited (0) 6 hours ago                          great_jepsen
b285532856e7   hello-world                    "/hello"                 7 hours ago   Exited (0) 7 hours ago                          dazzling_tharp
a813e8fe4ec6   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   7 hours ago   Up 7 hours                                      proxy-agent


In [43]:
!docker run -p 4000:80 --name my-app node-app:0.1

Unable to find image 'node-app:0.1' locally
docker: Error response from daemon: pull access denied for node-app, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.
See 'docker run --help'.


To test out the server, open a terminal window and type the following command:

```bash
curl http://localhost:4000
```

You should see the server respond with `Hello World`

The container will run as long as the initial terminal is running. If you want to stop the container, run the following command in the terminal to stop and remove the container:

```bash
docker stop my-app && docker rm my-app
```
After a few moments the container will stop. You should notice the cell above will complete execution.

#### Running the container in the background
If you want to the container to run in the background (not tied to the terminal's session), you need to specify the `-d` flag.
Now run the following command to start the container in the background

In [19]:
!docker run -p 4000:80 --name my-app -d node-app:0.1

docker: Error response from daemon: Conflict. The container name "/my-app" is already in use by container "7ca2f6f75d5b1b5ec04999c2e140aa748c313694f82ae705dfd4210f933b347d". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


Your container is now running in the background. You can check the status of your running container using `docker ps`

In [20]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED       STATUS       PORTS                  NAMES
7ca2f6f75d5b   b849727e7f46                   "node ./src/app.js"      6 hours ago   Up 6 hours   0.0.0.0:4000->80/tcp   my-app
a813e8fe4ec6   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   7 hours ago   Up 7 hours                          proxy-agent


Notice the container is running in the output of docker ps. You can look at the logs by executing `docker logs [container_id]`. 

In [23]:
# Note, your container id will be different
!docker logs 7ca2f6f75d5b

Server running at http://0.0.0.0:80/


You should see 
```bash
Server running at http://0.0.0.0:80/
```
If you want to follow the log's output as the container is running, use the `-f` option.

## Modify & Publish

Let's modify the application and push it to your Google Cloud Repository (gcr). After that you'll remove all local containers and images to simulate a fresh environment, and then pull and run your containers from gcr. This will demonstrate the portability of Docker containers.

### Edit `app.js`
Open the file `./src/app.js` with the text editor and replace "Hello World" with another string. Then build this new image. 

In [24]:
!docker build -t node-app:0.2 -f dockerfiles/intro.docker .

Sending build context to Docker daemon  90.62kB
Step 1/5 : FROM node:6
 ---> ab290b853066
Step 2/5 : WORKDIR /app
 ---> Using cache
 ---> dc1c34f69a79
Step 3/5 : ADD . /app
 ---> 989157b5c224
Step 4/5 : EXPOSE 80
 ---> Running in ad77c511ff07
Removing intermediate container ad77c511ff07
 ---> a78a735c14dd
Step 5/5 : CMD ["node", "./src/app.js"]
 ---> Running in fd284e6750c8
Removing intermediate container fd284e6750c8
 ---> 88cc49101e7e
Successfully built 88cc49101e7e
Successfully tagged node-app:0.2


Notice in `Step 2` of the output we are using an existing cache layer. From `Step 3` and on, the layers are modified because we made a change in `app.js`.

Run another container with the new image version. Notice how we map the host's port 8000 instead of 80. We can't use host port 4000 because it's already in use. 

In [25]:
!docker run -p 8000:80 --name my-app-2 -d node-app:0.2

d17f8449fbd4614c703d13d11b5537593aa7069c3d5792294df717a5f1868f1f


You can check that both container are running using `docker ps`.

In [26]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS          PORTS                  NAMES
d17f8449fbd4   node-app:0.2                   "node ./src/app.js"      14 seconds ago   Up 13 seconds   0.0.0.0:8000->80/tcp   my-app-2
7ca2f6f75d5b   b849727e7f46                   "node ./src/app.js"      6 hours ago      Up 6 hours      0.0.0.0:4000->80/tcp   my-app
a813e8fe4ec6   gcr.io/inverting-proxy/agent   "/bin/sh -c '/opt/bi…"   7 hours ago      Up 7 hours                             proxy-agent


And let's test boht containers using `curl` as before:

In [27]:
!curl http://localhost:8000

Hello virtual Axa HKG!!


In [28]:
!curl http://localhost:4000

Hello World!!


Recall, to stop a container running, you can execute the following command either in a terminal or (because they are running in the background) in a cell in this notebook. 

### Publish to gcr

Now you're going to push your image to the Google Container Registry (gcr). To push images to your private registry hosted by gcr, you need to tag the images with a registry name. The format is `[hostname]/[project-id]/[image]:[tag]`.

For gcr:

  * `[hostname]`= gcr.io
  * `[project-id]`= your project's ID
  * `[image]`= your image name
  * `[tag]`= any string tag of your choice. If unspecified, it defaults to "latest".

In [29]:
import os

PROJECT_ID = "qwiklabs-gcp-03-2974874276fb" # REPLACE WITH YOUR PROJECT NAME
os.environ["PROJECT_ID"] = PROJECT_ID

Let's tag `node-app:0.2`.

In [30]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED              SIZE
node-app                       0.2       88cc49101e7e   About a minute ago   884MB
node-app                       0.1       9ba864117cd6   4 minutes ago        884MB
<none>                         <none>    b849727e7f46   6 hours ago          884MB
hello-world                    latest    d1165f221234   7 weeks ago          13.3kB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   2 months ago         1.73GB
node                           6         ab290b853066   23 months ago        884MB


In [31]:
%%bash
docker tag node-app:0.2 gcr.io/${PROJECT_ID}/node-app:0.2

Now when we list our docker images we should see this newly tagged repository.

In [32]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED              SIZE
node-app                                       0.2       88cc49101e7e   About a minute ago   884MB
gcr.io/qwiklabs-gcp-03-2974874276fb/node-app   0.2       88cc49101e7e   About a minute ago   884MB
node-app                                       0.1       9ba864117cd6   4 minutes ago        884MB
<none>                                         <none>    b849727e7f46   6 hours ago          884MB
hello-world                                    latest    d1165f221234   7 weeks ago          13.3kB
gcr.io/inverting-proxy/agent                   <none>    fe507176d0e6   2 months ago         1.73GB
node                                           6         ab290b853066   23 months ago        884MB


Next, let's push this image to gcr.

In [33]:
%%bash
docker push gcr.io/${PROJECT_ID}/node-app:0.2

The push refers to repository [gcr.io/qwiklabs-gcp-03-2974874276fb/node-app]
654bfc8da7b5: Preparing
c17577722a70: Preparing
f39151891503: Preparing
f1965d3c206f: Preparing
a27518e43e49: Preparing
910d7fd9e23e: Preparing
4230ff7f2288: Preparing
910d7fd9e23e: Waiting
2c719774c1e1: Preparing
ec62f19bb3aa: Preparing
f94641f1fe1f: Preparing
2c719774c1e1: Waiting
4230ff7f2288: Waiting
ec62f19bb3aa: Waiting
f1965d3c206f: Layer already exists
a27518e43e49: Layer already exists
f39151891503: Layer already exists
910d7fd9e23e: Layer already exists
2c719774c1e1: Layer already exists
4230ff7f2288: Layer already exists
f94641f1fe1f: Layer already exists
ec62f19bb3aa: Layer already exists
c17577722a70: Pushed
654bfc8da7b5: Pushed
0.2: digest: sha256:58c8a33d420312642e70561743323498e4d1ea20ddc2f5396f6f85f6b5dce92a size: 2424


Check that the image exists in `gcr` by visiting the image registry Cloud Console. You can navigate via the console to `Navigation menu > Container Registry` or visit the url from the cell below:

In [34]:
%%bash
echo "http://gcr.io/${PROJECT_ID}/node-app"

http://gcr.io/qwiklabs-gcp-03-2974874276fb/node-app


### Test the published gcr image

Let's test this image. You could start a new VM, ssh into that VM, and install gcloud. For simplicity, we'll just remove all containers and images to simulate a fresh environment.

First, stop and remove all containers using `docker stop` and `docker rm`. **Be careful not to stop the container running this AI Notebook!**.

In [35]:
!docker stop my-app && docker rm my-app

my-app
my-app


In [36]:
!docker stop my-app-2 && docker rm my-app-2

my-app-2
my-app-2


Now remove the docker images you've created above using `docker rmi`.

In [42]:
!docker images

REPOSITORY                                     TAG       IMAGE ID       CREATED        SIZE
gcr.io/qwiklabs-gcp-03-2974874276fb/node-app   0.2       88cc49101e7e   15 hours ago   884MB
<none>                                         <none>    b849727e7f46   21 hours ago   884MB
gcr.io/inverting-proxy/agent                   <none>    fe507176d0e6   2 months ago   1.73GB


In [38]:
%%bash
docker rmi node-app:0.2
docker rmi gcr.io/${PROJECT_ID}/node-app:0.2
docker rmi node-app:0.1
docker rmi node:6 
docker rmi -f hello-world:latest

Untagged: node-app:0.2
Untagged: gcr.io/qwiklabs-gcp-03-2974874276fb/node-app:0.2
Untagged: gcr.io/qwiklabs-gcp-03-2974874276fb/node-app@sha256:58c8a33d420312642e70561743323498e4d1ea20ddc2f5396f6f85f6b5dce92a
Deleted: sha256:88cc49101e7e8f5437c300b6b5b6f974b27c397efc4d5a7d512f072e7018601e
Deleted: sha256:a78a735c14dd2c54ccfca1b8c9de81570e8fc6943b8fc3aae47726ebb9eb7ee9
Deleted: sha256:989157b5c22414df3dda94955dcd2932436ca0c4c7f927b92be7c3679933f574
Deleted: sha256:2e921b7a06b88116b9102304105ffa4af58c26a3138964d8ecb22e2c4dfed9b1
Untagged: node-app:0.1
Deleted: sha256:9ba864117cd6dcf2c5f8fae7a3153fe1a1b7c40932693f6083be294e070be5df
Deleted: sha256:15f2be766976e8a575d47387907b1d608d488e74feddc4a8267fdeafc14f6240
Deleted: sha256:7050de89e63941fe82babf89e992860b18e2c335f967b14b7e5284dc5d5cc9f8
Deleted: sha256:a15b8e71083ccffeb2dca365b21c2eb34c6ce24999fc0994696f9fb3e1dcb5ba
Untagged: node:6
Untagged: node@sha256:e133e66ec3bfc98da0440e552f452e5cdf6413319d27a2db3b01ac4b319759b3
Untagged: hello-

Confirm all images are removed with `docker images`.

In [39]:
!docker images

REPOSITORY                     TAG       IMAGE ID       CREATED        SIZE
<none>                         <none>    b849727e7f46   6 hours ago    884MB
gcr.io/inverting-proxy/agent   <none>    fe507176d0e6   2 months ago   1.73GB


At this point you should have a pseudo-fresh environment. Now, pull the image and run it.

In [40]:
%%bash
docker pull gcr.io/${PROJECT_ID}/node-app:0.2
docker run -p 4000:80 -d gcr.io/${PROJECT_ID}/node-app:0.2

0.2: Pulling from qwiklabs-gcp-03-2974874276fb/node-app
c5e155d5a1d1: Already exists
221d80d00ae9: Already exists
4250b3117dca: Already exists
3b7ca19181b2: Already exists
425d7b2a5bcc: Already exists
69df12c70287: Already exists
ea2f5386a42d: Already exists
d421d2b3c5eb: Already exists
f8bed8b456fd: Already exists
b9f3dc9a78ae: Pulling fs layer
b9f3dc9a78ae: Download complete
b9f3dc9a78ae: Pull complete
Digest: sha256:58c8a33d420312642e70561743323498e4d1ea20ddc2f5396f6f85f6b5dce92a
Status: Downloaded newer image for gcr.io/qwiklabs-gcp-03-2974874276fb/node-app:0.2
gcr.io/qwiklabs-gcp-03-2974874276fb/node-app:0.2
ac26c889fb12fcc898ed9e5d5d0f781bb37b48d50b62449b761bc90dd848f86e


You can check that it's running as expected using before:

In [41]:
!curl http://localhost:4000

Hello virtual Axa HKG!!


Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.